In [ ]:
import os
import pathlib
import logging
import tarfile

import numina.util.context as ctx
from numina.user.helpers import create_datamanager, load_observations
from numina.user.baserun import run_reduce
from numina.tests.testcache import download_cache


import matplotlib.pyplot as plt

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
basedir = pathlib.Path().resolve()

In [ ]:
tarball = 'MEGARA-cookbook-M15_LCB_HR-R-v1.tar.gz'
url = 'http://guaix.fis.ucm.es/~spr/megara_test/{}'.format(tarball)

downloaded = download_cache(url)

# Uncompress
with tarfile.open(downloaded.name, mode="r:gz") as tar:
    tar.extractall()

os.remove(downloaded.name)

In [ ]:
persist = False
    
if persist:
    reqfile = basedir / 'control_dump.yaml'
else:
    reqfile = basedir / 'control_v2.yaml'

datadir = basedir / 'data'

dm = create_datamanager(reqfile, basedir, datadir)

In [ ]:
obsresults = ["0_bias.yaml", "2_M15_modelmap.yaml",
         "4_M15_fiberflat.yaml", "6_M15_Lcbadquisition.yaml",
         "8_M15_reduce_LCB.yaml", "1_M15_tracemap.yaml",
         "3_M15_wavecalib.yaml", "5_M15_twilight.yaml",
         "7_M15_Standardstar.yaml"]

sessions, loaded_obs = load_observations(obsresults, is_session=False)
dm.backend.add_obs(loaded_obs)

In [ ]:
obsid = "0_bias"
task0 = run_reduce(dm, obsid)

In [ ]:
task0.result.qc

In [ ]:
master_bias = task0.result.master_bias.open()

In [ ]:
master_bias[0].header

In [ ]:
task0.request_runinfo

In [ ]:
obsid = "1_HR-R"
task1 = run_reduce(dm, obsid)

In [ ]:
print(task1.result.master_traces)

In [ ]:
task1.result.master_traces

In [ ]:
obsid = "3_HR-R"
task3 = run_reduce(dm, obsid)

In [ ]:
obsid = "4_HR-R"
task4 = run_reduce(dm, obsid)

In [ ]:
obsid = "5_HR-R"
task5 = run_reduce(dm, obsid)

In [ ]:
obsid = "6_HR-R"
task6 = run_reduce(dm, obsid)

In [ ]:
obsid = "7_HR-R"
task7 = run_reduce(dm, obsid)

In [ ]:
obsid = "8_HR-R"
task8 = run_reduce(dm, obsid)

In [ ]:
import yaml
with open('control_dump.yaml', 'w') as fd:
    datam = dm.backend.dump_data()
    yaml.dump(datam, fd)

In [ ]:
rss = task8.result.reduced_rss.open()

In [ ]:
plt.imshow(rss[0].data)

In [ ]:

from astropy.wcs import WCS
import numpy as np

In [ ]:
projection = WCS(rss['FIBERS'].header)

In [ ]:
def hexplot_helper(ax, img):
    import megaradrp.visualization as vis
    import megaradrp.datamodel as dm
    # This should be simplified
    rssdata = np.squeeze(img[0].data[:,2000])
    datamodel = dm.MegaraDataModel()
    fiberconf = datamodel.get_fiberconf(rss)
    x = np.empty((fiberconf.nfibers,))
    y = np.empty((fiberconf.nfibers,))
    # Key is fibid
    for _, fiber in sorted(fiberconf.fibers.items()):
        idx = fiber.fibid - 1
        x[idx] = fiber.x
        y[idx] = fiber.y
    scale = 0.443
    col = vis.hexplot(ax, x,y, rssdata, scale=scale)
    return col
    

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.15, 0.1, 0.8, 0.8], projection=projection)

ax.coords.grid()
ax.set_xlim([-6.5, 6.5])
ax.set_ylim([-6, 6])
col = hexplot_helper(ax, rss)